In [8]:
from openai import OpenAI

import time
from sklearn.metrics import accuracy_score

from basicmemnet import memnet


In [9]:
client = OpenAI(api_key = 'sk---') 
model = 'gpt-4o-mini'

In [10]:
md_train = memnet.DSL()
md_test = memnet.DSL()

md_train.import_gml("train_graph.gml")
md_test.import_gml("test_graph.gml")

test = md_test.get_graph()
train = md_train.get_graph()

In [11]:
parent_actions = ['task_1_k_cooking', 'task_2_k_cooking_with_bowls', 'task_3_k_pouring', 'task_4_k_wiping',
                  'task_5_k_cereals', 'task_6_w_hard_drive', 'task_7_w_free_hard_drive', 'task_8_w_hammering', 
                  'task_9_w_sawing']

In [12]:
examples_of_tasks = {}

for task in parent_actions:
    sub_graphs = md_train.get_stm_actions(action_attributes={"utterances": [task]})
    #print(task)
    examples = f"Examples of full graphs of {task}:\n"
    for gr in sub_graphs:
        #for node, data in gr.nodes(data=True):
        for u, v, link in gr.edges(data=True):
            relation = link['link_type']
            head = gr.nodes[u]['utterances'][0]
            tail = gr.nodes[v]['utterances'][0]
            if task not in [head, tail]:
                examples = examples + f"{head} {relation} {tail}, "
                # print(head, link['link_type'], tail)
        examples = examples[:-2] + ".\n"
    examples_of_tasks[task] = examples

In [13]:
instructions = """You are a knowledgeable assistant. Given a set of triples representing sub-actions of some parent actions, predict the most likely next sub-actions for the specific triples.

Below are examples of parent actions and their corresponding sequences of sub-actions. Each triple represents a head, relation, and tail from the knowledge graph describing the process.\n"""

In [14]:
ground_truths = []
predictions = []
n=0
for task in parent_actions:
    sub_graphs = md_test.get_stm_actions(action_attributes={"utterances": [task]})
    examples = examples_of_tasks[task]
    for gr in sub_graphs:
        prompt = f"\nWhat should be the next actions in the following triples: "
        current_ground_truth = []
        for u, v, link in gr.edges(data=True):
            relation = link['link_type']
            head = gr.nodes[u]['utterances'][0]
            tail = gr.nodes[v]['utterances'][0]
            if relation == 'has_next':
                prompt = prompt + f"\n({head} {relation} ?)"
                current_ground_truth.append(tail)
        # prompt = prompt + f"\nAnswer nothing but the sequence of {len(current_ground_truth)} space seperated next actions for each triple in the right order. Each member of the list has to be one of the 14 possible next actions: approach, cut, drink, hammer, hold, idle, lift, place, pour, retreat, saw, screw, stir, wipe."
        prompt = prompt + f"\nAnswer nothing but the sequence of space seperated next actions for each {len(current_ground_truth)} triple in the right order. Each member of the sequence has to be one of the following possible next actions: approach, cut, drink, hammer, hold, idle, lift, place, pour, retreat, saw, screw, stir, wipe."
        #print(n, task, "\n", instructions + examples + prompt)  
        # print("---------------------------------------------------")  
        response = client.chat.completions.create(
                    model=model,
                    messages=[{"role": "user", "content": instructions + examples + prompt}],
                    max_tokens=50,
                    temperature=0, 
                    top_p=0.9,       
                    n=1,             
                )
        answer = response.choices[0].message.content.strip()
        answer = answer.split()

        print(n, task)
        print("Truth:", current_ground_truth)
        ground_truths.extend(current_ground_truth)

        if len(answer) < len(current_ground_truth):
            print("-----------TOO FEW PREDICTIONS------------")
            answer.extend([-1]* (len(current_ground_truth)-len(answer)))
        elif len(answer) > len(current_ground_truth):
            print("----------TOO MANY PREDICTIONS------------")
            print("Raw answer:", answer)
            answer = answer[:len(current_ground_truth)]

        predictions.extend(answer)
        # print(prompt)
        
        print("Predictions:", answer)
        n+=1
        # time.sleep(18)

0 task_1_k_cooking
Truth: ['approach', 'hold', 'stir', 'retreat', 'approach', 'place', 'hold', 'pour', 'approach', 'place', 'retreat', 'lift', 'lift', 'approach', 'retreat', 'retreat']
Predictions: ['approach', 'approach', 'lift', 'place', 'approach', 'pour', 'approach', 'lift', 'retreat', 'stir', 'hold', 'approach', 'approach', 'retreat', 'place', 'hold']
1 task_1_k_cooking
Truth: ['approach', 'retreat', 'hold', 'lift', 'retreat', 'approach', 'hold', 'approach', 'lift', 'approach', 'retreat', 'retreat', 'place', 'stir', 'place', 'pour']
-----------TOO FEW PREDICTIONS------------
Predictions: ['approach', 'place', 'hold', 'approach', 'hold', 'retreat', 'approach', 'lift', 'retreat', 'hold', 'place', 'stir', 'lift', 'pour', 'lift', -1]
2 task_1_k_cooking
Truth: ['place', 'approach', 'hold', 'retreat', 'place', 'retreat', 'retreat', 'approach', 'approach', 'stir', 'pour', 'approach', 'hold', 'lift', 'lift', 'retreat']
----------TOO MANY PREDICTIONS------------
Raw answer: ['place', 'appr

In [122]:
print(instructions + examples + prompt)

You are a knowledgeable assistant. Given a set of triples representing sub-actions of some parent actions, predict the most likely next sub-actions for the specific triples.

Below are examples of parent actions and their corresponding sequences of sub-actions. Each triple represents a head, relation, and tail from the knowledge graph describing the process.
Examples of task_9_w_sawing:
retreat has_object saw, retreat has_next approach, retreat has_object woodenwedge, place has_object woodenwedge, place has_next retreat, lift has_object woodenwedge, lift has_next place, lift has_object saw, lift has_next saw, place has_object saw, place has_next retreat, approach has_object saw, approach has_next lift, lift has_object saw, lift has_next place, approach has_object woodenwedge, approach has_next hold, idle has_next approach, approach has_object saw, approach has_next lift, saw has_object saw, saw has_next place, idle has_next approach, hold has_object woodenwedge, hold has_next lift, ret

In [15]:
print(accuracy_score(ground_truths, predictions))

0.1561618062088429
